<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamA/blob/main/Abfragen_DWH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notwendige Schritte, um das DWH zu öffnen. Pfad muss ggf. angepasst oder auskommentiert werden. Dieser Abschnitt muss einmal ausgeführt werden, um sich mit dem DWH zu verbinden. Danach können die Abfragen
einzeln ausgeführt werden.



In [ ]:
import sqlite3
from sqlite3 import Error

from google.colab import drive
drive.mount('/content/drive')

# Path
material_path = "/content/drive/MyDrive/M3_DM_GrpA"

# path of datawarehouse
DB_DWH_PATH = material_path + f"/DWH_GrpA.db"

dwh_db = sqlite3.connect(DB_DWH_PATH)
dwh_cursor = dwh_db.cursor()

Mounted at /content/drive


Unabhängige Abfragen zur Nutzung

In [ ]:
# check list of tables
dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(dwh_cursor.fetchall())

[('patients_info',), ('county_info',), ('disease_info',), ('conditions_info',), ('careplans_info',), ('procedures_info',), ('medications_info',), ('observations_info',), ('devices_info',), ('imaging_studies_info',)]


In [ ]:
# check columns in a table (hier: county_info)
dwh_cursor.execute('PRAGMA table_info(' + "county_info" + ');')
dwh_cursor.fetchall()

[(0, 'STATE', 'STRING', 0, None, 0),
 (1, 'COUNTY', 'STRING', 0, None, 1),
 (2, 'POPULATION', 'INTEGER', 0, None, 0),
 (3, 'AREA_sqm', 'INTEGER', 0, None, 0),
 (4, 'DENSITY_per_sqm', 'INTEGER', 0, None, 0),
 (5, 'SEASIDE', 'INTEGER', 0, None, 0)]

In [ ]:
# Schreibe ein SQL Statement und lass die ersten 10 Zeilen ausgeben
# Bsp 1
dwh_cursor.execute("SELECT Id, BIRTHDATE, FIRST, LAST, MARITAL, GENDER from patients_info")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

('2f114e21-8b95-e9eb-2e9e-4b950b059cfa', '2002-05-05', 'Terisa250', 'Kreiger457', '', 'F')
('2812f4fc-6aba-0033-728b-7e569bb8adcc', '1942-05-24', 'Romeo514', 'Morar593', 'M', 'M')
('9b1a203a-6368-49a5-0c79-45c32729bc17', '2013-06-15', 'Jamaal34', 'Hessel84', '', 'M')
('52a250a9-24e4-ec9b-91d6-c38cf1b10fc0', '1997-11-16', 'Kenya524', 'Leuschke194', '', 'F')
('ad5525c0-fdab-94dc-018f-d7209c72bdef', '1965-08-28', 'Clark193', 'Little434', 'M', 'M')
('7f30313f-98e9-2cc2-68a5-6f9973f9ad5a', '1974-12-06', 'Simona661', 'Green467', 'M', 'F')
('f7dddcba-380e-7fd4-20b0-da086ce5dc3a', '1991-08-19', 'Moira454', 'Quitzon246', 'M', 'F')
('21e040b3-3c29-6042-1dba-da47dee67c18', '1952-12-25', 'Guillermo498', 'Cepeda588', 'M', 'M')
('b9f7feb0-45ac-55ed-4ee2-be12ad599416', '1980-02-24', 'Geralyn932', 'Steuber698', 'S', 'F')
('2b2d8855-c163-3838-a719-995a3cefca33', '1992-10-19', 'Gracia333', 'Lockman863', 'M', 'F')


In [ ]:
# Bsp 2
dwh_cursor.execute("SELECT PATIENT,CODE, DESCRIPTION STRING, BASE_COST from procedures_info")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

('52a250a9-24e4-ec9b-91d6-c38cf1b10fc0', 261352009, 'Face mask (physical object)', 4.67)
('2f114e21-8b95-e9eb-2e9e-4b950b059cfa', 261352009, 'Face mask (physical object)', 3.1)
('2812f4fc-6aba-0033-728b-7e569bb8adcc', 261352009, 'Face mask (physical object)', 1.33)
('7f30313f-98e9-2cc2-68a5-6f9973f9ad5a', 261352009, 'Face mask (physical object)', 3.56)
('9b1a203a-6368-49a5-0c79-45c32729bc17', 261352009, 'Face mask (physical object)', 1.95)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 261352009, 'Face mask (physical object)', 2.59)
('2812f4fc-6aba-0033-728b-7e569bb8adcc', 180325003, 'Electrical cardioversion', 44895.87)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 399208008, 'Plain chest X-ray (procedure)', 6211.16)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 371908008, 'Oxygen administration by mask (procedure)', 516.65)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 431182000, 'Placing subject in prone position (procedure)', 516.65)


In [ ]:
dwh_cursor.execute("SELECT COUNT(Id) AS Anzahl, REASONDESCRIPTION FROM careplans_info GROUP BY REASONDESCRIPTION")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

(738, 'COVID-19')
(905, 'Suspected COVID-19')


Abfrage: County, Density, Anzahl Covid

In [ ]:
dwh_cursor.execute("SELECT patients_info.COUNTY, county_info.DENSITY_per_sqm, COUNT(patients_info.Id), conditions_info.DESCRIPTION FROM conditions_info JOIN patients_info, county_info ON conditions_info.PATIENT = patients_info.Id AND patients_info.COUNTY = county_info.COUNTY WHERE conditions_info.DESCRIPTION = 'COVID-19' GROUP BY patients_info.COUNTY")
rows = dwh_cursor.fetchall()
for row in rows:
  print(row)

('Barnstable County', 595, 26, 'COVID-19')
('Berkshire County', 138, 21, 'COVID-19')
('Bristol County', 1070, 97, 'COVID-19')
('Dukes County', 216, 2, 'COVID-19')
('Essex County', 1697, 85, 'COVID-19')
('Franklin County', 101, 13, 'COVID-19')
('Hampden County', 757, 52, 'COVID-19')
('Hampshire County', 311, 17, 'COVID-19')
('Middlesex County', 2085, 201, 'COVID-19')
('Nantucket County', 345, 2, 'COVID-19')
('Norfolk County', 1889, 84, 'COVID-19')
('Plymouth County', 807, 76, 'COVID-19')
('Suffolk County', 14281, 84, 'COVID-19')
('Worcester County', 587, 105, 'COVID-19')
